# BBC dataset 
**Caution:** Many data valuation methods require training large number of models to get reliable estimates. **It is extremely slow**. We recommend using embeddings.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch

# Opendataval
from opendataval.dataloader import Register, DataFetcher, mix_labels, add_gauss_noise
from opendataval.dataval import (
    AME,
    DVRL,
    BetaShapley,
    DataBanzhaf,
    DataOob,
    DataShapley,
    InfluenceSubsample,
    KNNShapley,
    LavaEvaluator,
    LeaveOneOut,
    RandomEvaluator,
    RobustVolumeShapley,
)

from opendataval.experiment import ExperimentMediator

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


## [Step 1] Set up an environment
`ExperimentMediator` is a fundamental concept in establishing the `opendataval` environment. It empowers users to configure hyperparameters, including a dataset, a type of synthetic noise, and a prediction model. With  `ExperimentMediator`, users can effortlessly compute various data valuation algorithms.

The following code cell demonstrates how to set up `ExperimentMediator` with a pre-registered dataset and a prediction model.
- Dataset: bbc
- Model: transformer's DistilBertModel
- Metric: Classification accuracy

In [2]:
dataset_name = "bbc" 
train_count, valid_count, test_count = 1000, 100, 500
noise_rate = 0.1
noise_kwargs = {'noise_rate': noise_rate}
model_name = "BertClassifier"
metric_name = "accuracy"
train_kwargs = {"epochs": 2, "batch_size": 50}
device = torch.device('cuda')

exper_med = ExperimentMediator.model_factory_setup(
    dataset_name=dataset_name,
    cache_dir="../data_files/",  
    force_download=False,
    train_count=train_count,
    valid_count=valid_count,
    test_count=test_count,
    add_noise=mix_labels,
    noise_kwargs=noise_kwargs,
    train_kwargs=train_kwargs,
    device=device,
    model_name=model_name,
    metric_name=metric_name
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Base line model metric_name='accuracy': perf=0.9179999828338623


## [Step 2] Compute data values
`opendataval` provides various state-of-the-art data valuation algorithms. `ExperimentMediator.compute_data_values()` computes data values.

In [3]:
data_evaluators = [ 
    RandomEvaluator(),
#     LeaveOneOut(), # leave one out ## slow
    InfluenceSubsample(num_models=10), # influence function
#     DVRL(rl_epochs=10), # Data valuation using Reinforcement Learning ## inappropriate
#     KNNShapley(k_neighbors=valid_count), # KNN-Shapley ## inappropriate
#     DataShapley(gr_threshold=1.05, mc_epochs=300, cache_name=f"cached"), # Data-Shapley ## slow
#     BetaShapley(gr_threshold=1.05, mc_epochs=300, cache_name=f"cached"), # Beta-Shapley ## slow
    DataBanzhaf(num_models=10), # Data-Banzhaf
    AME(num_models=10), # Average Marginal Effects
    DataOob(num_models=10) # Data-OOB
#     LavaEvaluator(),
#     RobustVolumeShapley(mc_epochs=300)
]

In [4]:
%%time
# compute data values.
## Training multiple DistilBERT models is extremely slow. We recommend using embeddings.
exper_med = exper_med.compute_data_values(data_evaluators=data_evaluators)

Elapsed time RandomEvaluator(): 0:00:00.026117


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.69s/it]


Elapsed time InfluenceSubsample(num_models=10): 0:00:16.937929


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.28s/it]


Elapsed time DataBanzhaf(num_models=10): 0:00:12.857943


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.93s/it]


Elapsed time AME(num_models=10): 0:00:50.578566


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.71s/it]

Elapsed time DataOob(num_models=10): 0:00:27.150970
CPU times: user 1min 39s, sys: 7.2 s, total: 1min 46s
Wall time: 1min 47s


## [Step 3] Store data values

In [5]:
from opendataval.experiment.exper_methods import save_dataval

# Saving the results
output_dir = f"../tmp/{dataset_name}_{noise_rate=}/"
exper_med.set_output_directory(output_dir)
output_dir

'../tmp/bbc_noise_rate=0.1/'

In [6]:
exper_med.evaluate(save_dataval, save_output=True)

,indices,data_values
RandomEvaluator(),2106,0.562118
RandomEvaluator(),1300,0.831894
RandomEvaluator(),1235,0.541294
RandomEvaluator(),1474,0.052256
RandomEvaluator(),3,0.827014
...,...,...
DataOob(num_models=10),1915,1.0
DataOob(num_models=10),781,0.0
DataOob(num_models=10),2203,1.0
DataOob(num_models=10),1940,1.0
